In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import cross_val_score, StratifiedKFold
import xgboost as xgb
import plotly.express as px, seaborn as sns, matplotlib.pyplot as plt
sns.set_style('darkgrid')
from sklearn.metrics import make_scorer, cohen_kappa_score
import eli5
from eli5.sklearn import PermutationImportance
import warnings

warnings.simplefilter('ignore')

SyntaxError: invalid syntax (1311419210.py, line 1)

In [ ]:
pip install optuna

In [ ]:
import optuna

In [ ]:
path = '../input/child-mind-institute-problematic-internet-use/'

train = pd.read_csv(path + 'train.csv', index_col = 'id')
print("The train data has the shape: ",train.shape)
test = pd.read_csv(path + 'test.csv', index_col = 'id')
print("The test data has the shape: ",test.shape)
print("")
print("Total number of missing training values: ", train.isna().sum().sum())
data_dictionary = pd.read_csv(path + 'data_dictionary.csv')

In [ ]:
#- Xử lý dữ liệu phân loại (categorical data): Tìm và xử lý các cột không phải kiểu số (non-numeric columns) trong cả hai tập dữ liệu.
#- Chuẩn hóa giá trị của cột "season"
#+ Điền giá trị mặc định là 0 cho các giá trị bị thiếu (null/NaN)
#+ Chuyển đổi giá trị dạng text (chuỗi) trong cột "season" thành các số nguyên đại diện cho từng mùa.

In [ ]:
train_cat_columns = train.select_dtypes(exclude = 'number').columns

for season in train_cat_columns:
    train[season] = train[season].fillna(0)
    train[season] = train[season].replace({'Spring':1, 'Summer':2, 'Fall':3, 'Winter':4})

In [ ]:
test_cat_columns = test.select_dtypes(exclude = 'number').columns

for season in test_cat_columns:
    test[season] = test[season].fillna(0)
    test[season] = test[season].replace({'Spring':1, 'Summer':2, 'Fall':3, 'Winter':4})

In [ ]:
#Tìm các cột có chứa từ "PCIAT": Lấy danh sách tên cột:
PCIAT_cols = [val for val in train.columns[train.columns.str.contains('PCIAT')]]
# Đếm số lượng cột 
print('Number of PCIAT features = ' , len(PCIAT_cols))


In [ ]:
#Có 22 đặc trưng với 20 đặc trưng là những câu hỏi, 1 đặc trưng là total score và 1 đặc trưng là thời gian thực hiện.
#Ta có thể tạo đặc trưng sii (scaled impact index) dựa trên total score và các ngưỡng cụ thể của chúng:
#0-30: sii = 0
#31-49: sii = 1
#50-79: sii = 2
#80-100: sii = 3
#Có thể bỏ 20 đặc trưng và season, chỉ để lại total để làm regression. 
#Còn việc phân loại dựa trên sii ta có thể biến thành một bài toán classification với 4 lớp

In [ ]:
pd.set_option('display.max_colwidth', None)
# Lọc dữ liệu dựa trên điều kiện: Tạo một DataFrame questions chứa các dòng mà giá trị trong cột Field có chứa 'PCIAT-PCIAT'
questions = data_dictionary[data_dictionary.Field.str.contains('PCIAT-PCIAT')]
questions[['Field','Description']]

In [ ]:
corr = train[PCIAT_cols].corr()['PCIAT-PCIAT_Total'].sort_values(ascending = False)
corr = pd.DataFrame(corr)
corr.style.background_gradient(cmap='YlOrRd')
# Tính toán các đặc trưng có ảnh hưởng như nào đến với total, nếu đặc trưng có ảnh hưởng lớn tới total thì sẽ có màu đậm hơn

In [ ]:
sns.boxplot(train, x = 'PCIAT-PCIAT_Total').set_title('Boxplot of PCIAT Total Scores')

In [ ]:
print(train[train['PCIAT-PCIAT_Total']<=30].sii.value_counts())
print(train[(train['PCIAT-PCIAT_Total']>30) 
    & (train['PCIAT-PCIAT_Total']<50)].sii.value_counts())
print(train[(train['PCIAT-PCIAT_Total']>=50) 
    & (train['PCIAT-PCIAT_Total']<80)].sii.value_counts())
print(train[train['PCIAT-PCIAT_Total']>=80].sii.value_counts())

In [ ]:
train.sii.value_counts()

In [ ]:
PCIAT_cols.remove('PCIAT-PCIAT_Total')
train = train.drop(columns = PCIAT_cols)

In [ ]:
#Mô tả về cuộc thi: Cuộc thi liên quan đến việc đánh giá việc sử dụng internet quá mức ở trẻ em và thanh thiếu niên
#Phân tích dữ liệu: Ở đây, có 34 trường hơp có sii = 3.0, được coi là nặng trong việc "nghiện" internet

In [ ]:
vals = ['PIU = 0', 'PIU = 1','PIU = 2', 'PIU = 3']

for i in range(4):
    plt.figure()
    plot = sns.countplot(x = train[train.sii==i]['PreInt_EduHx-computerinternet_hoursday'])
    plot.set_title(vals[i])

có thể thấy một điều đáng chú ý rằng với PIU = 3 thì có 5 người lại không bị "nghiện" với sii = 0.0

In [ ]:
train = train.dropna(subset='sii')
#xóa các giá trị NaN trong cột sii

In [ ]:
corr = pd.DataFrame(train.corr()['PCIAT-PCIAT_Total'].sort_values(ascending = False))
corr.style.background_gradient(cmap='YlOrRd')
# tính toán và hiển thị mức độ ảnh hưởng của các feature đến total trong tập train 

In [ ]:
selection = corr[(corr['PCIAT-PCIAT_Total']>.1) | (corr['PCIAT-PCIAT_Total']<-.1)]
#lọc ra những feature mà có ảnh hưởng lớn tích cực hoặc tiêu cực đủ mạnh > 0.1 và < -0.1
selection = [val for val in selection.index]
selection.remove('PCIAT-PCIAT_Total')
selection.remove('sii')
selection.remove('Physical-BMI')
selection.remove('SDS-SDS_Total_Raw')
# xóa các cột không cần thiết đi
# bmi và sds bị xóa vị nó đã được tính lặp lại, total là cột kết quả nên sẽ bị xóa 

In [ ]:
null = train.isna().sum().sort_values(ascending = False).head(46)
null = pd.DataFrame(null)
null = null.rename(columns= {0:'Missing'})
null.style.background_gradient(cmap='YlOrRd')
# hiển thị missing value của tập train 

In [ ]:
half_missing = [val for val in train.columns[train.isnull().sum()>len(train)/2]]
half_missing
# do có nhiều feature bị thiếu dữ liệu nên ta sẽ chọn phương án là sử dụng những feature nào mà số lượng missing value < half

In [ ]:
selection = [i for i in selection if i not in half_missing]

In [ ]:
describe = train[selection].describe().T
describe = describe[['min','max']].sort_index()
describe.style.background_gradient(cmap='YlOrRd')
# tính giá trị max và min của từng feature được chọn để tính những cái bất thường, ví dụ như việc physical weight của một người = 0 là bất khả thi 

In [ ]:
train[selection].hist(figsize=(10,10), grid = True, color = 'chocolate')
plt.tight_layout()

In [ ]:
X = train[selection]
test = test[selection]
y = train['PCIAT-PCIAT_Total']

In [ ]:
def convert(scores):
    scores = np.array(scores)*1.252
    bins = np.zeros_like(scores)
    bins[scores <= 30] = 0
    bins[(scores > 30) & (scores < 50)] = 1
    bins[(scores >= 50) & (scores < 80)] = 2
    bins[scores >= 80] = 3
    return bins
#phần loại thành các class 0 , 1 , 2 , 3

In [ ]:
def quadratic_kappa(y_true, y_pred):
    y_true_cat = convert(y_true)
    y_pred_cat = convert(y_pred)
    return cohen_kappa_score(y_true_cat, y_pred_cat, weights='quadratic')
#Tính Quadratic Weighted Kappa (QWK) giữa giá trị thực (y_true) và giá trị dự đoán (y_pred).
kappa_scorer = make_scorer(quadratic_kappa, greater_is_better=True)
#Tạo một thước đo (scorer) tùy chỉnh để sử dụng trong huấn luyện và đánh giá mô hình. trong đó Dùng hàm quadratic_kappa làm thước đo. 
# greater_is_better=True: Càng cao, mô hình càng tốt.

In [ ]:
def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
    }

    model = xgb.XGBRegressor(**params)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X, y, cv=skf, scoring=kappa_scorer)
    return np.mean(scores)  # Maximize mean QWK score across folds


In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Lấy siêu tham số tốt nhất
best_params = study.best_params
print("Best parameters:", best_params)

In [ ]:
#params = {'max_depth': 3, 'n_estimators': 59, 'learning_rate': 0.074, 'subsample': 0.6, 'colsample_bytree': 0.92}
#model = xgb.XGBRegressor(**best_params)

In [ ]:
model = xgb.XGBRegressor(**best_params)

In [ ]:
skf = StratifiedKFold(n_splits=10)
scores = cross_val_score(model, X, y, cv=skf, scoring=kappa_scorer)
#Đánh giá mô hình bằng cross-validation.
print("QWK Scores:", scores)
print("Mean QWK Score:", np.mean(scores))

In [ ]:
model.fit(X,y)
# huấn luyện mô hình XGBoost với đầu vào X là tập dữ liệu đặc trưng còn y là đầu ra, hay ở đây là total 
feature_imp = pd.Series(model.feature_importances_,index=X.columns).sort_values(ascending=False)
# tính toán tầm quan trọng của các đặc trưng dựa trên  tần suất hoặc mức độ giảm lỗi của các đặc trưng 
# khi được sử dụng để phân chia dữ liệu trong cây quyết định (decision tree)
feature_imp

In [ ]:
sns.barplot(x=feature_imp, y=feature_imp.index)
plt.xlabel('Feature Importance Score')
plt.title("Feature Importances")
plt.show()

In [ ]:
# Permutation Importance (Tầm quan trọng hoán vị) để đánh giá mức độ ảnh hưởng
# của từng đặc trưng (feature) đến kết quả của mô hình
# Hoạt động bằng cách xáo trộn (hoán vị) các giá trị của từng đặc trưng và quan sát sự giảm hiệu suất.
perm = PermutationImportance(model, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names = X.columns.tolist())
# 0.1101 ± 0.0102: trong đó 0.1101 cho biết mức độ quan trọng của feature với mô hình còn 0.0102 cho thấy mức độ sai số 

In [ ]:
model.fit(X,y)
preds = model.predict(test)
preds = convert(preds) 
# convert raw scores to sii categories if using regressor
preds = pd.Series(preds)
preds.index = test.index
preds.to_csv('submission.csv')

In [ ]:
submission = pd.read_csv('submission.csv')
submission